In [10]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


##What is a relational Database?

"A database is a means of storing information in such a way that information can be retrieved from it. In simplest terms, a relational database is one that presents information in tables with rows and columns. A table is referred to as a relation in the sense that it is a collection of objects of the same type (rows). Data in a table can be related according to common keys or concepts, and the ability to retrieve related data from a table is the basis for the term relational database."

https://docs.oracle.com/javase/tutorial/jdbc/overview/database.html

##How does a database compare to a spreadsheet?

###Table

Each table is like a separate spreadsheet (or spreadsheet tab).


###Example Table: Fruit

In the example below, we create the fruit table, insert data into it and then display that data. The text inside of the c.execute statements is Structured Query Language (SQL), which is used to create, read, update and delete tables and the data inside of them.

In [11]:
import sqlite3 #sqlite3 is a database library
from prettytable import from_db_cursor #used for printing tables/queries

print ("Fruit")
conn = sqlite3.connect(':memory:') #create and get connection to in-memory temporary database
c = conn.cursor() #cursors are used for executing commands and selecting data from database

# Create table
c.execute('DROP TABLE IF EXISTS fruit')
c.execute('''CREATE TABLE fruit
             ( fruit_id   INTEGER PRIMARY KEY
             , fruit_name TEXT
             , price      NUMERIC)''')

# Insert data
c.execute("INSERT INTO fruit VALUES (1, 'apple', 0.88)")
c.execute("INSERT INTO fruit VALUES (2, 'banana', 0.34)")
c.execute("INSERT INTO fruit VALUES (3, 'orange', 1.07)")
c.execute("INSERT INTO fruit VALUES (4, 'forbidden', NULL)")
c.execute("INSERT INTO fruit VALUES (5, '', NULL)")
c.execute("INSERT INTO fruit VALUES (NULL, NULL, 0)")

conn.commit() # Save (commit) the changes

c.execute("SELECT * FROM fruit ORDER BY fruit_id")  #select all table rows
print ( from_db_cursor(c) ) #print query


Fruit
+----------+------------+-------+
| fruit_id | fruit_name | price |
+----------+------------+-------+
|    1     |   apple    |  0.88 |
|    2     |   banana   |  0.34 |
|    3     |   orange   |  1.07 |
|    4     | forbidden  |  None |
|    5     |            |  None |
|    6     |    None    |   0   |
+----------+------------+-------+


###Column

Unlike spreadsheet columns, database columns each have a specific type and can only store certain types of data. The most common types include: TEXT, NUMERIC, INTEGER and REAL. Unlike spreadsheets, columns cannot be merged. In the above example, the columns are: `fruit_id`, `fruit_name` and `price`. The `fruit_id` column is a primary key which means values for it cannot be repeated so that a value uniquely identifies its row. 

see https://www.sqlite.org/datatype3.html

###Row

Like spreadsheets, tables are organized into rows of data. The example above has 6 rows of data.

###Field

A column for a particular row is called a field. 

###NULL

If a field has no data, it contains the special value `NULL`. This is different than the empty string: "" and the number: 0. Primary keys usually cannot be NULL. 

In [12]:
print ("Fruit with NULL price")
c.execute('''SELECT * FROM fruit 
             WHERE price is NULL 
             ORDER BY fruit_id''') 
pt = from_db_cursor(c)
print (pt)

print ("\nFruit with price = 0")
c.execute('''SELECT * FROM fruit 
             WHERE price = 0
             ORDER BY fruit_id''') 
pt = from_db_cursor(c)
print (pt)

print ('\nFruit with name = ""')
c.execute('''SELECT * FROM fruit 
             WHERE fruit_name = "" 
             ORDER BY fruit_id''') 
pt = from_db_cursor(c)
print (pt)


Fruit with NULL price
+----------+------------+-------+
| fruit_id | fruit_name | price |
+----------+------------+-------+
|    4     | forbidden  |  None |
|    5     |            |  None |
+----------+------------+-------+

Fruit with price = 0
+----------+------------+-------+
| fruit_id | fruit_name | price |
+----------+------------+-------+
|    6     |    None    |   0   |
+----------+------------+-------+

Fruit with name = ""
+----------+------------+-------+
| fruit_id | fruit_name | price |
+----------+------------+-------+
|    5     |            |  None |
+----------+------------+-------+


##Querying data with Structure Query Language (SQL)

SQL Queries are where we see the true power of relational databases. Using SQL we can ask powerful questions to filter, join and aggregate data. see: https://en.wikipedia.org/wiki/SQL#Queries

##Simple queries from one table


All queries begin with `SELECT`. 

Next, you specify which columns and formulas to select, separated by commas. For all column use the `*` character. You can  specify formulas in your select such as:

`SELECT part_id, part_price + part_tax AS total_part_price FROM parts`. 

Note the use of `AS` to create an alias for the formula:  `part_price + part_tax`

Next, you specify the table(s) your query will read data from. 


###Joins

The examples above have all only used a single table. Joins allow us to combine data from more than one table.  

###Normalization

You might ask why we need more than one table. Why not just store all the data in one table? The short answer is that data repeated more than one place is easy to mess up by placing it into a conflicting state. In the example below, you can see that salesperson_id `2` has different phone numbers and emails for different rows. It would be difficult to tell which phone number or email is the right one.

In [13]:
print("This should be two tables, not one since it allows for inconsistent data.")
c.execute('DROP TABLE IF EXISTS sale_representative	')
c.execute('''CREATE TABLE sale_representative	
             ( sale_id INTEGER PRIMARY KEY
             , sale_amount NUMERIC
             , sale_tax NUMERIC
             , representative_id INTEGER
             , representative_phone_number INTEGER
             , representative_email TEXT)''')
# Insert data

c.execute('''INSERT INTO sale_representative 
               ("sale_id", "sale_amount", "sale_tax", "representative_id", "representative_phone_number", "representative_email") 
             VALUES (NULL, 1.10, .11, 1, 5554441111, "employee1@fake.com") ''')

c.execute('''INSERT INTO sale_representative VALUES (NULL, 2.20, .22, 2, 5554442222, "employee2@fake.com") ''')
c.execute('''INSERT INTO sale_representative VALUES (NULL, 3.30, .33, 2, 5554442223, "employee2b@fake.com") ''')

# Save (commit) the changes
conn.commit()

#print table
from prettytable import from_db_cursor 
c.execute("SELECT * FROM sale_representative	 ORDER BY sale_id") 
pt = from_db_cursor(c)
print (pt)

This should be two tables, not one since it allows for inconsistent data.
+---------+-------------+----------+-------------------+-----------------------------+----------------------+
| sale_id | sale_amount | sale_tax | representative_id | representative_phone_number | representative_email |
+---------+-------------+----------+-------------------+-----------------------------+----------------------+
|    1    |     1.1     |   0.11   |         1         |          5554441111         |  employee1@fake.com  |
|    2    |     2.2     |   0.22   |         2         |          5554442222         |  employee2@fake.com  |
|    3    |     3.3     |   0.33   |         2         |          5554442223         | employee2b@fake.com  |
+---------+-------------+----------+-------------------+-----------------------------+----------------------+


In [14]:
print("Now that we have two tables with separate responsibilities, data isn't duplicated")
c.execute('DROP TABLE IF EXISTS representative')
c.execute('''CREATE TABLE representative 
             ( representative_id INTEGER PRIMARY KEY
             , representative_phone_number INTEGER
             , representative_email TEXT
             )''')

c.execute('''INSERT INTO representative ("representative_id", "representative_phone_number", "representative_email") 
             VALUES (1, 5554441111, "employee1@fake.com") ''')

c.execute('''INSERT INTO representative ("representative_id", "representative_phone_number", "representative_email") 
             VALUES (2, 5554442222, "employee2@fake.com") ''')

c.execute('DROP TABLE IF EXISTS sale')
c.execute('''CREATE TABLE sale 
             ( sale_id INTEGER PRIMARY KEY
             , sale_amount NUMERIC
             , sale_tax NUMERIC
             , representative_fk INTEGER
             , FOREIGN KEY(representative_fk) REFERENCES representative(representative_id)
             )''')
# Insert data


c.execute('''INSERT INTO sale (sale_id, sale_amount, sale_tax, representative_fk) VALUES (NULL, 1.10, .11, 1) ''')
c.execute('''INSERT INTO sale (sale_id, sale_amount, sale_tax, representative_fk) VALUES (NULL, 2.20, .22, 2) ''')
c.execute('''INSERT INTO sale (sale_id, sale_amount, sale_tax, representative_fk) VALUES (NULL, 3.30, .33, 2) ''')

# Save (commit) the changes
conn.commit()

print("\nrepresentative")
c.execute("SELECT * FROM representative    ORDER BY representative_id") 
print ( from_db_cursor(c) )

print("\nsale")
c.execute("SELECT * FROM sale ORDER BY sale_id") 
print ( from_db_cursor(c) )

Now that we have two tables with separate responsibilities, data isn't duplicated

representative
+-------------------+-----------------------------+----------------------+
| representative_id | representative_phone_number | representative_email |
+-------------------+-----------------------------+----------------------+
|         1         |          5554441111         |  employee1@fake.com  |
|         2         |          5554442222         |  employee2@fake.com  |
+-------------------+-----------------------------+----------------------+

sale
+---------+-------------+----------+-------------------+
| sale_id | sale_amount | sale_tax | representative_fk |
+---------+-------------+----------+-------------------+
|    1    |     1.1     |   0.11   |         1         |
|    2    |     2.2     |   0.22   |         2         |
|    3    |     3.3     |   0.33   |         2         |
+---------+-------------+----------+-------------------+


In [15]:
print("We can still join the data, but duplicate data (and inconsistent data) no longer possible")

c.execute('''SELECT sale.sale_id, representative.representative_id, representative_phone_number, representative_email FROM sale 
             INNER JOIN representative on sale.representative_fk = representative.representative_id 
             ORDER BY sale_id ASC''') 
print ( from_db_cursor(c) )

We can still join the data, but duplicate data (and inconsistent data) no longer possible
+---------+-------------------+-----------------------------+----------------------+
| sale_id | representative_id | representative_phone_number | representative_email |
+---------+-------------------+-----------------------------+----------------------+
|    1    |         1         |          5554441111         |  employee1@fake.com  |
|    2    |         2         |          5554442222         |  employee2@fake.com  |
|    3    |         2         |          5554442222         |  employee2@fake.com  |
+---------+-------------------+-----------------------------+----------------------+


In [16]:
c.close

<function Cursor.close>